In [7]:
import sqlite3

old_db_conn = sqlite3.connect('./backup/old.db')
new_db_conn = sqlite3.connect('./kwu-lecture-recommendation-v2-updating.db')

old_cursor = old_db_conn.cursor()
new_cursor = new_db_conn.cursor()


old_cursor.execute('''
SELECT year, semester, lecNumber, lecIsPNP, lecIsEngeneering, lecRequirementClass, 
       lecIsTBL, lecIsPBL, lecIsSeminar, lecIsSmall, lecIsConvergence, 
       lecIsTeamTeach, lecIsFocus, lecIsArt, lecOverview, lecEvaluateRatio
FROM LectureTable
''')

rows = old_cursor.fetchall()

for row in rows:
    year = row[0]
    semester = row[1]
    lecNumber = row[2]
    lecIsPNP = row[3]
    lecIsEngineering = row[4]
    lecRequirementClass = row[5]
    lecIsTBL = row[6]
    lecIsPBL = row[7]
    lecIsSeminar = row[8]
    lecIsSmall = row[9]
    lecIsConvergence = row[10]
    lecIsTeamTeach = row[11]
    lecIsFocus = row[12]
    lecIsArt = row[13]
    lecOverview = row[14]
    lecEvaluateRatio = row[15]

    new_cursor.execute('''
    SELECT lectureID FROM LectureList
    WHERE year = ? AND semester = ? AND lecNumber = ?
    ''', (year, semester, lecNumber))

    lecture_id_row = new_cursor.fetchone()

    if lecture_id_row:
        lectureID = lecture_id_row[0]

        new_cursor.execute('''
        INSERT INTO LectureDetailData (lectureID, isPNP, isEngineering, isTBL, isPBL, 
                                       isSeminar, isSmall, isConvergence, 
                                       isTeamTeaching, isFocus, isArt, 
                                       Overview, evaluationRatio)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (lectureID, lecIsPNP, lecIsEngineering, lecIsTBL, lecIsPBL,
              lecIsSeminar, lecIsSmall, lecIsConvergence,
              lecIsTeamTeach, lecIsFocus, lecIsArt,
              lecOverview, lecEvaluateRatio))


new_db_conn.commit()


old_db_conn.close()
new_db_conn.close()

print("done")

done


In [5]:
# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')#
cursor = conn.cursor()


q = """CREATE VIEW LectureDetailView AS
SELECT
    l.lectureID,
    l.year,
    l.semester,
    l.lecNumber,
    l.lecName,
    l.lecProfessor,
    l.lecClassification,
    l.lecTheme,
    l.lecCredit,
    l.lecTime,
    l.lecWeekTime,
    l.lecClassroom,
    l.isLecClose,
    ld.takenPeople1yearsAgo,
    ld.takenPeople2yearsAgo,
    ld.takenPeople3yearsAgo,
    ld.ForeignLanguage,
    ld.percentageOfOnline,
    ld.isPNP,
    ld.isEngineering,
    ld.isTBL,
    ld.isPBL,
    ld.isSeminar,
    ld.isSmall,
    ld.isConvergence,
    ld.isTeamTeaching,
    ld.isFocus,
    ld.isExperimentDesign,
    ld.isELearning,
    ld.isArt,
    ld.representCompetency,
    ld.learningGoalNmethod,
    ld.Overview,
    ld.evaluationRatio,
    ld.mainBook,
    ld.scheduleNcontent,
    le.everytimeURL,
    le.star,
    le.assignmentAmount,
    le.teamPlayAmount,
    le.gradeAmount,
    le.reviewSummary,
    le.checkAttend,
    le.testNum
FROM
    LectureList l
LEFT JOIN
    LectureDetailData ld ON l.lectureID = ld.lectureID
LEFT JOIN
    LectureEverytimeData le ON l.lectureID = le.lectureID;"""



conn.execute(q)
conn.commit()  # 변경사항을 저장합니다.
conn.close()   # 연결을 닫습니다.

OperationalError: view LectureDetailView already exists

In [3]:
import csv
import sqlite3


new_db_conn = sqlite3.connect('kwu-lecture-recommendation-v4 copy 3.db')
new_cursor = new_db_conn.cursor()

def convert_to_yn(value):
    return 1 if value.lower() == 'checked' else 0

# CSV 파일 열 이름과 데이터베이스 필드 매핑
field_mapping = {
    '년도학기': 'year',
    '학정번호': 'lecNumber',
    '강의소개': 'Overview',
    '영어강의 비율': 'evaluationRatio',
    'TBL강의': 'isTBL',
    'PBL강의': 'isPBL',
    '세미나강의': 'isSeminar',
    '소규모강의': 'isSmall',
    '융합강의': 'isConvergence',
    '팀티칭강의': 'isTeamTeaching',
    '집중이수제강의': 'isFocus',
    '예체능': 'isArt'
}

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
cursor = conn.cursor()

# CSV 파일 읽기
with open('lecture_plan_24_2(1).csv', 'r', encoding='cp949') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        # 학정번호가 있는 경우에만 처리
        if row['학정번호']:
            # 년도와 학기 분리
            year_semester = row['년도학기'].split('-')
            # 고정 값
            year = 24
            semester = '2학기'
            # LectureList 테이블에서 lectureID 조회
            new_cursor.execute('''
            SELECT lectureID FROM LectureList
            WHERE year = ? AND semester = ? AND lecNumber = ?
            ''', (year, semester, row['학정번호']))

            lecture_id_row = cursor.fetchone()

            if lecture_id_row:
                lectureID = lecture_id_row[0]

                # LectureDetailData 테이블에 삽입할 데이터 준비
                insert_data = {
                    'lectureID': lectureID,
                    'Overview': row.get('강의소개', ''),
                    'evaluationRatio': row.get('영어강의 비율', ''),
                    'isTBL': convert_to_yn(row.get('TBL강의', '')),
                    'isPBL': convert_to_yn(row.get('PBL강의', '')),
                    'isSeminar': convert_to_yn(row.get('세미나강의', '')),
                    'isSmall': convert_to_yn(row.get('소규모강의', '')),
                    'isConvergence': convert_to_yn(row.get('융합강의', '')),
                    'isTeamTeaching': convert_to_yn(row.get('팀티칭강의', '')),
                    'isFocus': convert_to_yn(row.get('집중이수제강의', '')),
                    'isArt': convert_to_yn(row.get('예체능', ''))
                }

                # SQL 쿼리 생성
                columns = ', '.join(insert_data.keys())
                placeholders = ', '.join(['?' for _ in insert_data])
                sql = f'''
                INSERT INTO LectureDetailData ({columns})
                VALUES ({placeholders})
                '''

                # 데이터 삽입 실행
                cursor.execute(sql, list(insert_data.values()))

# 변경사항 저장 및 연결 종료
conn.commit()
conn.close()

print("데이터 가져오기가 완료되었습니다.")

데이터 가져오기가 완료되었습니다.


In [1]:
import csv
import sqlite3

def convert_to_yn(value):
    return 1 if value.lower() == 'checked' else 0

# CSV 파일 열 이름과 데이터베이스 필드 매핑
field_mapping = {
    '년도학기': 'year',
    '학정번호': 'lecNumber',
    '강의소개': 'Overview',
    '영어강의 비율': 'evaluationRatio',
    'TBL강의': 'isTBL',
    'PBL강의': 'isPBL',
    '세미나강의': 'isSeminar',
    '소규모강의': 'isSmall',
    '융합강의': 'isConvergence',
    '팀티칭강의': 'isTeamTeaching',
    '집중이수제강의': 'isFocus',
    '예체능': 'isArt',
    '원격 비율': 'percentageOfOnline'
}

try:
    # 데이터베이스 연결
    conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
    cursor = conn.cursor()

    # CSV 파일 읽기
    with open('lecture_plan_24_2(1).csv', 'r', encoding='cp949') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            # 학정번호가 있는 경우에만 처리
            if row['학정번호']:
                # 년도와 학기 분리
                year_semester = row['년도학기'].split('-')
                # 고정 값
                year = 24
                semester = '2학기'

                # LectureList 테이블에서 lectureID 조회
                cursor.execute('''
                SELECT lectureID FROM LectureList
                WHERE year = ? AND semester = ? AND lecNumber = ?
                ''', (year, semester, row['학정번호']))
                
                lecture_id_row = cursor.fetchone()
                
                if lecture_id_row:
                    lectureID = lecture_id_row[0]
                    
                    # LectureDetailData 테이블에 삽입할 데이터 준비
                    insert_data = {
                        'lectureID': lectureID,
                        'Overview': row.get('교과목 개요', ''),
                        'evaluationRatio': row.get('강의 비율', ''),
                        'isTBL': convert_to_yn(row.get('TBL강의', '')),
                        'isPBL': convert_to_yn(row.get('PBL강의', '')),
                        'isSeminar': convert_to_yn(row.get('세미나강의', '')),
                        'isSmall': convert_to_yn(row.get('소규모강의', '')),
                        'isConvergence': convert_to_yn(row.get('융합강의', '')),
                        'isTeamTeaching': convert_to_yn(row.get('팀티칭강의', '')),
                        'isFocus': convert_to_yn(row.get('집중이수제강의', '')),
                        'isArt': convert_to_yn(row.get('예체능', '')),
                        'percentageOfOnline': row.get('원격 비율', '')
                    }
                    
                    # SQL 쿼리 생성
                    columns = ', '.join(insert_data.keys())
                    placeholders = ', '.join(['?' for _ in insert_data])
                    sql = f'''
                    INSERT INTO LectureDetailData ({columns})
                    VALUES ({placeholders})
                    '''
                    
                    # 데이터 삽입 실행
                    cursor.execute(sql, list(insert_data.values()))
                    print(f"Inserted data for lecture: {row['학정번호']}")
                else:
                    print(f"No matching lectureID found for: {row['학정번호']}")

    # 변경사항 저장
    conn.commit()
    print("데이터 가져오기가 완료되었습니다.")

except sqlite3.Error as e:
    print(f"데이터베이스 오류: {e}")
except csv.Error as e:
    print(f"CSV 파일 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류: {e}")
finally:
    # 연결 종료
    if conn:
        conn.close()

Inserted data for lecture: 0000-1-8128-01
Inserted data for lecture: 0000-1-8128-02
Inserted data for lecture: I000-1-3362-01
Inserted data for lecture: I000-1-3362-02
Inserted data for lecture: I000-1-3362-03
Inserted data for lecture: I000-1-3362-04
Inserted data for lecture: I000-1-3362-05
Inserted data for lecture: I000-1-3362-06
Inserted data for lecture: I000-1-3362-07
Inserted data for lecture: I000-1-3362-08
Inserted data for lecture: I000-1-3362-09
Inserted data for lecture: I000-1-3362-10
Inserted data for lecture: I000-1-3362-11
Inserted data for lecture: I000-1-3362-12
Inserted data for lecture: I000-1-3362-13
Inserted data for lecture: I000-1-3362-14
Inserted data for lecture: 1000-1-3362-01
Inserted data for lecture: 1000-1-3362-02
Inserted data for lecture: 1000-1-3362-03
Inserted data for lecture: 1000-1-3362-04
Inserted data for lecture: 1000-1-3362-05
Inserted data for lecture: 1000-1-3362-06
Inserted data for lecture: 3000-1-3362-01
Inserted data for lecture: 3000-1-

Inserted data for lecture: 0000-2-4970-08
Inserted data for lecture: 0000-2-4970-09
Inserted data for lecture: 0000-2-4970-10
Inserted data for lecture: 0000-2-4970-11
Inserted data for lecture: 0000-2-4970-12
Inserted data for lecture: 0000-2-4969-01
Inserted data for lecture: 0000-2-6020-01
Inserted data for lecture: 0000-2-7724-01
Inserted data for lecture: 0000-2-0819-01
Inserted data for lecture: 0000-2-0819-02
Inserted data for lecture: 0000-2-0819-03
Inserted data for lecture: 0000-2-0819-04
Inserted data for lecture: 0000-2-0819-05
Inserted data for lecture: 0000-3-1942-01
Inserted data for lecture: 0000-3-8120-01
Inserted data for lecture: 0000-1-3416-01
Inserted data for lecture: 0000-1-3416-02
Inserted data for lecture: 0000-1-3416-03
Inserted data for lecture: 0000-1-3416-04
Inserted data for lecture: 0000-1-3416-05
Inserted data for lecture: 0000-1-3414-01
Inserted data for lecture: 0000-1-9884-01
Inserted data for lecture: 0000-1-9884-02
Inserted data for lecture: 0000-1-

In [4]:
import csv
import sqlite3

new_db_conn = sqlite3.connect('kwu-lecture-recommendation-v4 copy.db')

new_cursor = new_db_conn.cursor()

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4.db')
cursor = conn.cursor()

# isLecClose 값을 변환하는 함수
def convert_is_lec_close(value):
    return '1' if value.upper() == 'TRUE' else '0'

# CSV 파일 읽기 및 데이터베이스에 삽입
with open('yn_final_kwu-timetable5.csv', 'r', encoding='cp949') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    for row in csvreader:
        # 데이터 추출 및 변환
        lecNumber = row['학정번호']
        lecName = row['과목명']
        lecProfessor = row['담당교수']
        lecClassification = row['이수']
        lecTheme = row['테마']
        lecCredit = row['학점']
        lecTime = row['강의시간']
        lecWeekTime = row['시수']
        lecClassroom = row['강의실']
        isLecClose = convert_is_lec_close(row['폐강'])

        # 고정 값
        year = '24'
        semester = '2학기'
        new_cursor.execute('''
        SELECT lectureID FROM LectureList
        WHERE year = ? AND semester = ? AND lecNumber = ?
        ''', (year, semester, lecNumber))

        lecture_id_row = new_cursor.fetchone()

        if lecture_id_row:
            lectureID = lecture_id_row[0]
            # 데이터베이스에 삽입 또는 업데이트
            cursor.execute('''
            INSERT OR REPLACE INTO LectureList 
            (lecNumber, lecName, lecProfessor, lecClassification, lecTheme, 
            lecCredit, lecTime, lecWeekTime, lecClassroom, isLecClose, 
            year, semester)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (lecNumber, lecName, lecProfessor, lecClassification, lecTheme,
                lecCredit, lecTime, lecWeekTime, lecClassroom, isLecClose,
                year, semester))

# 변경사항 커밋 및 연결 종료
conn.commit()
conn.close()

print("Data insertion completed.")

KeyError: '과목명'